## Create a session

In [1]:
from sqlalchemy import (Column, ForeignKey, Integer, MetaData, String, Table,
                        create_engine)
from sqlalchemy.orm import (Mapped, declarative_base, mapped_column,
                            relationship, sessionmaker)
from models import *
# Create a session
Session = sessionmaker(bind=engine)
session = Session()


create database

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
concrete_scenario = session.query(TCEnhancedConcreteScenario).\
                            join(TestCase).\
                            filter(TestResult.id == 128).first()
print(concrete_scenario.path)

In [16]:


import os

# Specify the directory to list files from
directory_path = '/home/weidonghu/Desktop/log files'

# List to hold the file names
file_names = []

# List all files in the directory
for file in os.listdir(directory_path):
    if os.path.isfile(os.path.join(directory_path, file)):
        if "data" in file:
            file_names.append("/home/weidonghu/Documents/"+file)

print(file_names)
trs = session.query(TestResult).all()
trs_log = []
for tr in trs:
    trs_log.append(str(tr.log_file))

print(trs_log)
for file in file_names:
    if file not in trs_log:
        print(file)


['/home/weidonghu/Documents/data_2024_30_05_19_33_38.csv', '/home/weidonghu/Documents/data_2024_30_05_16_20_13.csv', '/home/weidonghu/Documents/data_2024_30_05_16_36_50.csv', '/home/weidonghu/Documents/data_2024_30_05_16_57_13.csv', '/home/weidonghu/Documents/data_2024_30_05_15_57_54.csv', '/home/weidonghu/Documents/data_2024_04_06_11_38_37.csv', '/home/weidonghu/Documents/data_2024_30_05_18_31_28.csv', '/home/weidonghu/Documents/data_2024_04_06_10_50_15.csv', '/home/weidonghu/Documents/data_2024_30_05_17_41_33.csv', '/home/weidonghu/Documents/data_2024_30_05_17_58_06.csv', '/home/weidonghu/Documents/data_2024_30_05_17_28_38.csv', '/home/weidonghu/Documents/data_2024_30_05_16_46_04.csv', '/home/weidonghu/Documents/data_2024_30_05_17_37_53.csv', '/home/weidonghu/Documents/data_2024_30_05_18_27_46.csv', '/home/weidonghu/Documents/data_2024_04_06_10_31_27.csv', '/home/weidonghu/Documents/data_2024_30_05_19_15_04.csv', '/home/weidonghu/Documents/data_2024_30_05_18_49_06.csv', '/home/weidon

In [49]:
import shutil
import os

def copy_file(src_path, dest_folder, overwrite=False):
    # Ensure the destination directory exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
        print(f"Created destination directory: {dest_folder}")
    
    # Construct the destination file path
    dest_path = os.path.join(dest_folder, os.path.basename(src_path))
    
    # Check if the file already exists
    if os.path.exists(dest_path):
        if overwrite:
            print(f"Overwriting existing file: {dest_path}")
        else:
            print(f"File already exists and overwrite is set to False. Skipping copy.")
            return
    
    # Copy the file
    try:
        shutil.copy(src_path, dest_path)
        print(f"File copied from {src_path} to {dest_path}")
    except Exception as e:
        print(f"Error copying file: {e}")

import json

def create_json(data, dest_folder, json_filename):
    # Ensure the destination directory exists
    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
        print(f"Created destination directory: {dest_folder}")
    
    # Construct the destination file path for the JSON file
    json_path = os.path.join(dest_folder, json_filename)
    
    # Write the dictionary data to the JSON file
    try:
        with open(json_path, 'w') as json_file:
            json.dump(data, json_file, indent=4)
        print(f"JSON file created at: {json_path}")
    except Exception as e:
        print(f"Error creating JSON file: {e}")

home_directory = os.path.expanduser('~')
results = session.query(TestResult).all()
for idx, result in enumerate(results):
    scenario = None
    regulation = None
    req_behavior = None
    log_file = None
    tc = None
    # move the scenario, the log file into one folder, make a text about the regulation and the req_behavior
    tc = session.query(TriggeringCondition).\
        join(TCEnhancedFunctionalScenario).\
        join(TCEnhancedLogicalScenario).\
        join(TCEnhancedConcreteScenario).join(TestCase).\
        join(TestResult).filter(TestResult.id == result.id).first().triggering_condition_name
    regulation = session.query(Regulation).join(RegulationTestCase).join(TestCase).\
        join(TestResult).filter(TestResult.id == result.id).all()
    regulation = [reg.regulation_description for reg in regulation]
    req_behavior = session.query(RequiredBehavior).join(RegulationRequiredBehavior).join(Regulation).join(RegulationTestCase).join(TestCase).\
        join(TestResult).filter(TestResult.id == result.id).all()
    req_behavior = [rb.behavior_name for rb in req_behavior]
    h_behavior = session.query(HazardousBehavior).join(TestResultHazardousBehavior).join(TestResult).filter(TestResult.id == result.id).all()
    h_behavior = [hb.behavior_description for hb in h_behavior]
    # files
    tcc_scenario = session.query(TCEnhancedConcreteScenario).join(TestCase).\
        join(TestResult).filter(TestResult.id == result.id).first().path
    log_file = result.log_file
    replay_file = log_file.replace("data", "scenario").replace('.csv','.log')
    data = {
        "TC": tc,
        "Regulations": regulation,
        "Python evaluation methods": req_behavior,
        "Test Result": "Pass" if result.test_result else "Failed",
        "Hazardous Behaviors": h_behavior
    }
    # move files
    dest_folder = "/home/weidonghu/Desktop/Scenario data/scenarios/scenario_"+str(idx+1)+"/"
    copy_file(log_file, dest_folder)
    copy_file(replay_file, dest_folder)
    copy_file(tcc_scenario.replace("~", home_directory), dest_folder)
    create_json(data, dest_folder, "info.json")


Created destination directory: /home/weidonghu/Desktop/Scenario data/scenarios/scenario_1/
File copied from /home/weidonghu/Documents/data_2024_30_05_15_48_41.csv to /home/weidonghu/Desktop/Scenario data/scenarios/scenario_1/data_2024_30_05_15_48_41.csv
File copied from /home/weidonghu/Documents/scenario_2024_30_05_15_48_41.log to /home/weidonghu/Desktop/Scenario data/scenarios/scenario_1/scenario_2024_30_05_15_48_41.log
File copied from /home/weidonghu/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/nl3/nc5/tcc/Cyclist_crosses_from_the_roadside/tcc33/tcc33.scenic to /home/weidonghu/Desktop/Scenario data/scenarios/scenario_1/tcc33.scenic
JSON file created at: /home/weidonghu/Desktop/Scenario data/scenarios/scenario_1/info.json
Created destination directory: /home/weidonghu/Desktop/Scenario data/scenarios/scenario_2/
File copied from /home/weidonghu/Documents/data_2024_30_05_15_50_32.csv to /home/weidonghu/Desktop/Scenario data/scenarios/scenario_2/data_2024_30_05_15_50_32.csv

In [48]:
regulation = session.query(Regulation).all()
d = list()
for reg in regulation:
    rbs = session.query(RequiredBehavior).join(RegulationRequiredBehavior).join(Regulation).filter(Regulation.id == reg.id).all()
    rbs = [rb.behavior_name for rb in rbs]
    x = {
        "Regulation":reg.regulation_description,
        "Python method's name": rbs
    }
    d.append(x)

dest_folder = "/home/weidonghu/Desktop/Scenario data/scenarios/"
create_json(d, dest_folder, "Regulation_mapping")


Created destination directory: /home/weidonghu/Desktop/Scenario data/scenarios/
JSON file created at: /home/weidonghu/Desktop/Scenario data/scenarios/Regulation_mapping


In [38]:
a = 0
b = "Pass" if a else "Failed"
print(b)

Failed


In [ ]:
"""Open all tcls"""

import subprocess, os, time
home_dir = os.path.expanduser('~')

tcfs={ 
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf1.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf2.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf3.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf4.md",
}

tcls = session.query(TCEnhancedLogicalScenario).all()

for tcl in tcls:
    path = str(tcl.path).replace("~",home_dir)
    if os.path.exists(path):
        print(path)
        time.sleep(0.1)
        subprocess.run(['code', path])




In [52]:
"""Find thesis scenarios"""
import subprocess, os, time, pprint
home_dir = os.path.expanduser('~')

tcls = session.query(TCEnhancedLogicalScenario).all()

print(len(tcls))

tcls_thesis = []
for tcl in tcls:
    if "Cyclist_crosses_from_the_roadside" in str(tcl.path):
        indexs = [1,3,5,7,9,11,13,15]
        for i in indexs:
            if f"tcl{i}.scenic" in str(tcl.path):
                tcls_thesis.append(tcl.path)
    if "Front_cyclist_moves_towards_the_road_edge_and_halt" in str(tcl.path):
        indexs = [1,3,5,7,9,11,13,15]
        for i in indexs:
            if f"tcl{i}.scenic" in str(tcl.path):
                tcls_thesis.append(tcl.path)
    if "Multiple_vehicles_cut_in_to_ego_path_in_a_short_time" in str(tcl.path):
        indexs = [1,3,5,7]
        for i in indexs:
            if f"tcl{i}.scenic" in str(tcl.path):
                tcls_thesis.append(tcl.path)
    if "Traffic_jam_in_front_when_neighbor_lane_is_free" in str(tcl.path):
        indexs = [2,4,6,8]
        for i in indexs:
            if f"tcl{i}.scenic" in str(tcl.path):
                tcls_thesis.append(tcl.path)

print(len(tcls_thesis))


tccs = session.query(TCEnhancedConcreteScenario).all()
print(len(tccs))

tccs = session.query(TCEnhancedConcreteScenario).\
                            join(TCEnhancedLogicalScenario).\
                            filter(TCEnhancedLogicalScenario.path.in_(tcls_thesis)).order_by(TCEnhancedLogicalScenario.path).all()
                            
print(len(tccs))

test_results = session.query(TestResult).\
                    join(TestCase).\
                    join(TCEnhancedConcreteScenario).\
                    join(TCEnhancedLogicalScenario).\
                    filter(TCEnhancedLogicalScenario.path.in_(tcls_thesis)).order_by(TCEnhancedLogicalScenario.path).all()

qualitative = dict()
        
# for result in test_results:
#     tc = session.query(TriggeringCondition).\
#         join(TCEnhancedFunctionalScenario).\
#         join(TCEnhancedLogicalScenario).\
#         join(TCEnhancedConcreteScenario).join(TestCase).\
#         join(TestResult).filter(TestResult.id == result.id).first()
#     hbs = [hb.behavior_description for hb in result.hazardous_behaviors]
#     for hb in hbs:
#         if not result.test_result:
#             if tc.triggering_condition_name not in qualitative:
#                 qualitative[tc.triggering_condition_name] = {hb}
#             else:
#                 qualitative[tc.triggering_condition_name].add(hb)

for result in test_results:
    tccs = session.query(TCEnhancedConcreteScenario).\
        join(TestCase).\
        join(TestResult).filter(TestResult.id == result.id).first().path
    tcid = session.query(TestCase).\
            join(TestResult).filter(TestResult.id == result.id).first().id
    log = result.log_file
    tc = session.query(TriggeringCondition).\
        join(TCEnhancedFunctionalScenario).\
        join(TCEnhancedLogicalScenario).\
        join(TCEnhancedConcreteScenario).join(TestCase).\
        join(TestResult).filter(TestResult.id == result.id).first()
    hbs = [hb.behavior_description for hb in result.hazardous_behaviors]
    if not result.test_result:
        if tc.triggering_condition_name not in qualitative:
            qualitative[tc.triggering_condition_name] = [(tcid,(result.test_result),log,tccs,hbs,)]
        else:
            qualitative[tc.triggering_condition_name].append((tcid,(result.test_result),log,tccs,hbs))

pprint.pprint(qualitative)


48
24
192
96
{'Cyclist crosses from the roadside': [(72,
                                        False,
                                        '/home/weidonghu/Documents/data_2024_04_06_09_54_45.csv',
                                        '~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/nl1/nc2/tcc/Cyclist_crosses_from_the_roadside/tcc10/tcc10.scenic',
                                        ['Ego collided with road user.']),
                                       (80,
                                        False,
                                        '/home/weidonghu/Documents/data_2024_30_05_18_54_39.csv',
                                        '~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/nl1/nc2/tcc/Cyclist_crosses_from_the_roadside/tcc14/tcc14.scenic',
                                        ['Ego collided with road user.',
                                         'Ego failed to reach the specified '
                                         'desti

In [ ]:
import subprocess, os, time
home_dir = os.path.expanduser('~')

tcfs={ 
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf1.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf2.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf3.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf4.md",
}

concrete_scenarios = session.query(TCEnhancedConcreteScenario).\
                            join(TCEnhancedLogicalScenario).\
                            join(TCEnhancedFunctionalScenario).\
                            filter(TCEnhancedFunctionalScenario.path.in_(tcfs)).all()

for tcc in concrete_scenarios:
    path = str(tcc.path).replace("~",home_dir)
    if os.path.exists(path):
        print(path)
        time.sleep(0.1)
        subprocess.run(['code', path])



In [ ]:
tcfs={ 
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Front_cyclist_moves_towards_the_road_edge_and_halt/tcf1.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Front_cyclist_moves_towards_the_road_edge_and_halt/tcf2.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Front_cyclist_moves_towards_the_road_edge_and_halt/tcf3.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Front_cyclist_moves_towards_the_road_edge_and_halt/tcf4.md",
}



test_results = session.query(TestResult).\
                        filter(TestResult.test_result == False).\
                        join(TestCase).\
                        join(TCEnhancedConcreteScenario).\
                        join(TCEnhancedLogicalScenario).\
                        join(TCEnhancedFunctionalScenario).\
                        filter(TCEnhancedFunctionalScenario.path.in_(tcfs)).\
                        order_by(TestResult.id).all()

concrete_scenario = session.query(TCEnhancedConcreteScenario).\
                            join(TCEnhancedLogicalScenario).\
                            join(TCEnhancedFunctionalScenario).\
                            filter(TCEnhancedFunctionalScenario.path.in_(tcfs)).\
                            join(TestCase).\
                            join(TestResult).\
                            filter(TestResult.log_file == "/home/weidonghu/Documents/data_2024_29_05_15_27_37.csv").first()

print(concrete_scenario.path)
for res in test_results:
    print((res.log_file, res.test_result))

In [ ]:
concrete_scenario = session.query(TCEnhancedConcreteScenario).\
                            join(TCEnhancedLogicalScenario).\
                            join(TCEnhancedFunctionalScenario).\
                            join(TestCase).\
                            join(TestResult).\
                            filter(TestResult.id == 128).first()

test_result = session.query(TestResult).filter(TestResult.id == 128).first()

regs = session.query(Regulation).\
                join(RegulationTestCase).\
                join(TestCase).\
                join(TestResult).\
                filter(TestResult.id == 128).all()

print(concrete_scenario.path)
print(test_result.test_result)
for reg in regs:
    print(reg.regulation_description)
print(test_result.hazardous_behaviors)

In [ ]:
concrete_scenario = session.query(TCEnhancedConcreteScenario).\
                            join(TCEnhancedLogicalScenario).\
                            join(TCEnhancedFunctionalScenario).\
                            join(TestCase).\
                            join(TestResult).\
                            filter(TestResult.log_file == "/home/weidonghu/Documents/data_2024_29_05_15_27_37.csv").first()
test_result = session.query(TestResult).filter(TestResult.log_file == "/home/weidonghu/Documents/data_2024_29_05_15_27_37.csv").first()

regs = session.query(Regulation).\
                join(RegulationTestCase).\
                join(TestCase).\
                join(TestResult).\
                filter(TestResult.log_file == "/home/weidonghu/Documents/data_2024_29_05_15_27_37.csv").all()

print(concrete_scenario.path)
print(test_result.test_result)
for reg in regs:
    print(reg.regulation_description)
print(test_result.hazardous_behaviors)

### tc -> regulations

In [ ]:
tcs = session.query(TriggeringCondition).all()
for tc in tcs:
    regs = session.query(Regulation).\
                join(RegulationTestCase).\
                join(TestCase).\
                join(TCEnhancedConcreteScenario).\
                join(TCEnhancedLogicalScenario).\
                join(TCEnhancedFunctionalScenario).\
                join(TriggeringCondition).\
                filter(TriggeringCondition.triggering_condition_name == tc.triggering_condition_name).all()
    print("TC: ", tc.triggering_condition_name)
    print()
    for reg in regs:
        print(reg.regulation_description)
    print("---------------------------")

In [ ]:
import subprocess, os, time
home_dir = os.path.expanduser('~')

tcfs={ 
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Front_cyclist_moves_towards_the_road_edge_and_halt/tcf1.md",
    # "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Front_cyclist_moves_towards_the_road_edge_and_halt/tcf2.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Front_cyclist_moves_towards_the_road_edge_and_halt/tcf3.md",
    # "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Front_cyclist_moves_towards_the_road_edge_and_halt/tcf4.md",
}

tcls={
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/nl2/tcl/Front_cyclist_moves_towards_the_road_edge_and_halt/tcl5/tcl5.scenic",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/nl2/tcl/Front_cyclist_moves_towards_the_road_edge_and_halt/tcl6/tcl6.scenic",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/nl4/tcl/Front_cyclist_moves_towards_the_road_edge_and_halt/tcl13/tcl13.scenic",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/nl4/tcl/Front_cyclist_moves_towards_the_road_edge_and_halt/tcl14/tcl14.scenic"
}

first = True
for tcf in tcfs:
    path = str(tcf).replace("~",home_dir)
    if first:
        time.sleep(0.1)
        subprocess.run(['code','-n', path])
        first = False
    else:
        time.sleep(0.1)
        subprocess.run(['code', path])

logical_scenarios = session.query(TCEnhancedLogicalScenario).\
                            filter(TCEnhancedLogicalScenario.path.in_(tcls)).\
                            order_by(TCEnhancedLogicalScenario.path).all()


for tcl in logical_scenarios:
    path = str(tcl.path).replace("~",home_dir)
    if os.path.exists(path):
        print(path)
        time.sleep(0.1)
        subprocess.run(['code', path])

concrete_scenarios = session.query(TCEnhancedConcreteScenario).\
                            join(TCEnhancedLogicalScenario).\
                            filter(TCEnhancedLogicalScenario.path.in_(tcls)).\
                            order_by(TCEnhancedConcreteScenario.path).all()


for tcc in concrete_scenarios:
    path = str(tcc.path).replace("~",home_dir)
    if os.path.exists(path):
        print(path)
        time.sleep(0.1)
        subprocess.run(['code', path])



In [ ]:
import subprocess, os, time
home_dir = os.path.expanduser('~')

tcfs={ 
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Multiple_vehicles_cut_in_to_ego_path_in_a_short_time/tcf1.md",
    "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/tcf/Multiple_vehicles_cut_in_to_ego_path_in_a_short_time/tcf2.md",
    # "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf3.md",
    # "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf2/tcf/Traffic_jam_in_front_when_neighbor_lane_is_free/tcf4.md",
}
first = True
for tcf in tcfs:
    path = str(tcf).replace("~",home_dir)
    if first:
        time.sleep(0.1)
        subprocess.run(['code','-n', path])
        first = False
    else:
        time.sleep(0.1)
        subprocess.run(['code', path])

logical_scenarios = session.query(TCEnhancedLogicalScenario).\
                            join(TCEnhancedFunctionalScenario).\
                            filter(TCEnhancedFunctionalScenario.path.in_(tcfs)).order_by(TCEnhancedLogicalScenario.path).all()

for tcl in logical_scenarios:
    path = str(tcl.path).replace("~",home_dir)
    if os.path.exists(path):
        print(path)
        time.sleep(0.1)
        subprocess.run(['code', path])
        py_path = path.replace(".scenic",".py")
        subprocess.run(['code', py_path])

concrete_scenarios = session.query(TCEnhancedConcreteScenario).\
                            join(TCEnhancedLogicalScenario).\
                            join(TCEnhancedFunctionalScenario).\
                            filter(TCEnhancedFunctionalScenario.path.in_(tcfs)).order_by(TCEnhancedConcreteScenario.path).all()


for tcc in concrete_scenarios:
    path = str(tcc.path).replace("~",home_dir)
    if os.path.exists(path):
        print(path)
        time.sleep(0.1)
        subprocess.run(['code', path])



In [ ]:
current_file_path = "~/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/nl1/nc1/tcc/Front_cyclist_moves_towards_the_road_edge_and_halt/tcc1/tcc1.scenic"

required_behaviors = session.query(RequiredBehavior).\
                            join(RegulationRequiredBehavior).\
                            join(Regulation).\
                            join(RegulationTestCase).\
                            join(TestCase).\
                            join(TCEnhancedConcreteScenario).\
                            filter(TCEnhancedConcreteScenario.path == current_file_path).all()
for required_behavior in required_behaviors:
    print(required_behavior.behavior_name)

In [ ]:
import os,re

def extract_number(filename):
    match = re.search(r'(\d+)\.scenic', filename)
    if match:
        return int(match.group(1))
    else:
        return float('inf')

nl = list()
path = "/home/weidong/Tools/Scenic/scenic_projects/nominal_scenario_catalog/nf1/"
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        # find nls
        if re.compile(r"tcc\d+\.scenic").match(filename) and "Cyclist_crosses_from_the_roadside" in dirpath:
            nl_path = os.path.join(dirpath, filename)
            nl.append(nl_path)

nl.sort(key=extract_number)
nl

In [ ]:
import json 

with open('regulations.json') as f:
    # Load JSON data
    data = json.load(f)
    regulations = data['regulations']
    regus = [(r['regulation'], r['required_behavior']) for r in data['regulations']]
    rbhbs = [(r['required_behavior'],r['hazardous_behavior']) for r in data['regulations']]

rb_hb = dict()
for rbhb_tuple in rbhbs:
    for idx, rb in enumerate(rbhb_tuple[0]):
        if rb in rb_hb:
            if rb_hb[rb] != rbhb_tuple[1][idx]:
                print(rb)
        rb_hb[rb] =  rbhb_tuple[1][idx]



In [ ]:

from sqlalchemy import (Column, ForeignKey, Integer, MetaData, String, Table,
                        create_engine)
from sqlalchemy.orm import (Mapped, declarative_base, mapped_column,
                            relationship, sessionmaker)
from models import *
import os,re


# Create a session
Session = sessionmaker(bind=engine)
session = Session()
rb_sql = [RequiredBehavior(behavior_name=rb) for rb in rb_hb.keys()]
rb_hb

### Add nominal scenarios

In [ ]:
# 
functional_nominal_scenario = {}

functional_nominal_scenario_1 = FunctionalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_1/functional_scenario_description.md")
# 
logical_nominal_scenario_1 = LogicalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_1/logical_scenario_1/logical_scenario_1.scenic")
# 
concrete_nominal_scenario_1 = ConcreteNominalScenario(path="nominal_scenario_catalog/nominal_scenario_1/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py")
# 
functional_nominal_scenario_1.logical_nominal_scenario_items.append(logical_nominal_scenario_1)
logical_nominal_scenario_1.concrete_nominal_scenario_items.append(concrete_nominal_scenario_1)
# 
session.add(functional_nominal_scenario_1)
session.commit()

In [ ]:
# Nominal
functional_nominal_scenario_1 = FunctionalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/functional_scenario_description.md")
# 
logical_nominal_scenario_1 = LogicalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/logical_scenario_1.py")
# 
concrete_nominal_scenario_1 = ConcreteNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py")
concrete_nominal_scenario_2 = ConcreteNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/concrete_scenario_2/concrete_scenario_2.py")
# 
functional_nominal_scenario_1.logical_nominal_scenario_items.append(logical_nominal_scenario_1)
logical_nominal_scenario_1.concrete_nominal_scenario_items.append(concrete_nominal_scenario_1)
logical_nominal_scenario_1.concrete_nominal_scenario_items.append(concrete_nominal_scenario_2)
# 
session.add(functional_nominal_scenario_1)
session.commit()

Add TCs

In [ ]:
TCs = [
    {'TC':'Parking vehicle on the roadside with opening door', 'FI':'Information processing-lidar-based object classification, Plan-behavior prediction'},
    {'TC':'E-scooter drives on motor lane', 'FI':'Plan-situation understanding'},
    {'TC':'Vehicle from the right merging lane without right of way brakes too late', 'FI':'Plan-prediction'},
    {'TC':'Workers unload a truck on the road', 'FI':'Information processing-object detection, Plan-trajectory prediction'},
    {'TC':'Pedestrian halts sharply in front of the crosswalk', 'FI':'Plan-trajectory prediction'},
    {'TC':'Front cyclist leaves the lane', 'FI':'Information processing - object detection, Plan - situation understanding'},
    {'TC':'Multiple vehicles cut in to ego path in a short time', 'FI':'Plan - situation understanding & decision making'},
    {'TC':'Front vehicle drives on lane markings', 'FI':'Plan-situation understanding'},
    {'TC':'Many vehicles park along the road side', 'FI':'Information access - offline map backup, Information processing - localization, road mark detection, Plan-situation understanding, prediction'},
    {'TC':'Traffic jam in front when neighbor lane is free', 'FI':'Plan - situation understanding + prediction'},
    {'TC':'Front vehicle halts on the lane with warning flasher on', 'FI':'Plan - situation understanding'},
    {'TC':'Cyclist crosses from the roadside', 'FI':'Information reception - FOV,, Plan - Prediction'},
    {'TC':'Oncoming vehicle overtakes a halting vehicle', 'FI':'Plan-situation understanding'},
    {'TC':'Scooter/cyclist drives on opposite lane', 'FI':'Plan-situation understanding'},
    {'TC':'Traffic jam on the opposite lane', 'FI':'Information processing-object separation, Plan-situation understanding, prediction'},
    {'TC':'Halting vehicles on both directions narrow the road', 'FI':'Plan-situation understanding'},
    {'TC':'Front cyclist moves towards the road edge and halt', 'FI':'Plan-behavior interpretation & prediction'},
]


for TC in TCs:
    session.add(TriggeringCondition(triggering_condition_name=TC['TC'], functional_insufficiency=TC['FI'], functional_insufficiency_origin='expert'))
session.commit()

### Add TC scenarios

In [ ]:
tc_enhanced_functional_scenario_1 = TCEnhancedFunctionalScenario(path = "tc_scenario_catalog/Cyclist_crosses_from_roadside/functional_scenario_description.md",
                                                                 exist_non_complient = 0,
                                                                 functional_nominal_scenario_id = 1,
                                                                 triggering_condition_id = 12
                                                                 )
# 
tc_enhanced_logical_scenario_1 = TCEnhancedLogicalScenario(path = "tc_scenario_catalog/Cyclist_crosses_from_roadside/logical_scenario_1/logical_scenario_1.scenic",
                                                           logical_nominal_scenario_id = 1)

tc_enhanced_functional_scenario_1.tc_enhanced_logical_scenario_items.append(tc_enhanced_logical_scenario_1)

session.add(tc_enhanced_functional_scenario_1)
session.commit()

In [ ]:


tc_enhanced_concrete_scenario_1 = TCEnhancedConcreteScenario(path = "tc_scenario_catalog/Cyclist_crosses_from_roadside/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py",
                                                             concrete_nominal_scenario_id = 1,
                                                             tc_enhanced_logical_scenario_id = 1)
session.add(tc_enhanced_concrete_scenario_1)
session.commit()

### Add regulations

In [ ]:
regulations = [
("The AD-Function shall adjust the speed of the AD-Vehicle to applicable speed limits.",("ego_speed_limit")),
("the AD-Function shall not perform harsh brake without a compelling reason.", ("ego_safe_deceleration")),
("The AD-Function shall only initiate a regular lane change if there is a reason for a lane change.", ("ego_lane_keeping")),
("The AD-Function shall reach the specified destination, in the absence of any accident.",("ego_reach_destination")),
("Under urban and rural driving conditions, the AD-Function shall avoid a collision, up to a speed of 60 km/h, with an unobstructed cyclist crossing with a lateral speed component of not more than 15 km/h in front of the vehicle. This shall be ensured independently from the specific manoeuvre the AD-Function is undertaking.", 
("collision")),
("In general, the AD-Function shall drive normally.",
("ego_safe_deceleration")),
("When overtaking pedestrians, cyclists or personal light electric vehicles in built-up areas, the AD-Function shall keep at least 1.5 m lateral distance. If necessary, the AD-Function shall wait. At intersections and junctions, the minimum lateral distance of 1.5m to cyclists does not apply, if cyclists have overtaken the AD-Vehicle waiting there on the right or have come to a standstill next to them. ",
("ego_overtake_safe_lateral_distance")),
("The AD-Function shall not overtake in unclear traffic situations.",("ego_overtake")),
("While the AD-Vehicle is not at standstill, and operating in speed range up to 60 km/h, the AD-Function shall adapt the speed to adjust the distance to a lead vehicle in the same lane to be equal or greater than the minimum following distance.",
("safe_longitudinal_distance")),
("Only if the AD-Function can oversee, that there will be no obstruction of oncoming traffic during the entire overtaking process, the AD-Function may overtake.",
("ego_safe_overtake_oncoming_traffic"))
("Only if the AD-Function drives with considerably higher speed than the vehicle being overtaken, the AD-Function may overtake.",("ego_safe_overtake_higher_speed")),
("The AD-Function shall overtake on the left, except defined otherwise in separate requirements.",("ego_overtaking_on_the_left")),
("During overtaking, AD-Function shall keep enough lateral distance to other road users.", ("ego_overtake_safe_lateral_distance")),
("When being overtaken, the AD-Function shall not increase speed.",("ego_being_overtaken")),
("In \"CCFhol: Car-to-Car Front Head-On Lane change\" scenarios, the AD-Function shall reduce speed by at least 20 km/h (original speed vs. impact speed), in case a collision cannot be avoided.",
("ego_emergency_braking")),
("If, according to traffic regulations, the AD-Vehicle is allowed to proceed or otherwise has priority, the AD-Function shall give up this right if required by the traffic situation.",
("ego_react_to_pedetrian")),              
]

for regu in regulations:
    regulation = Regulation(regulation_description=regu[0])

    tc_enhanced_concrete_scenario_1.test_case = test_case_1

test_case_1.required_behaviors.append(required_behavior_1)

session.add(test_case_1)
session.add(tc_enhanced_concrete_scenario_1)
session.commit()

### Add required behaviors

In [ ]:

requiredNhazard_behaviors = [("collision","Ego caused collision with target."),
                      ("exceed_acceleration","Ego exceeded specified acceleration."),
                      ("minimum_distance","Ego violated minimum distance with target."),
                      ("ego_overtake","Ego overtook target unsafely."),
                      ("safe_longitudinal_distance", "Ego violated safe longitudinal distance with front vehicle."),
                      ("ego_drive_normally", "Ego drive abnormally."),
                      ("ego_overtake_safe_lateral_distance", "Ego violated safe lateral distance while overtaking target."),
                      ("ego_safe_overtake_oncoming_traffic", "Ego conduct unsafe overtake despite oncoming traffic."),
                      ("ego_safe_overtake_higher_speed", "Ego conduct unsafe overtake despite without considerably higher speed than target."),
                      ("ego_being_overtaken", "Ego increased speed while being overtaken."),
                      ("ego_emergency_braking", "Ego failed at emergency braking."),
                      ("ego_overtaking_on_the_left", "Ego overtook on the right."),
                      ("ego_react_to_pedetrian", "Ego failed at stop in front of pedestrian."),
                      ("ego_safe_deceleration","Ego deceleration was lower than threshold."),
                      ("ego_lane_keeping", "Ego deviated from its original lane."),
                      ("ego_reach_destination", "Ego failed to reach destination"),
                      ("ego_speed_limit", "Ego's speed exceeded the speed limit.")
                    ]

for behavior in requiredNhazard_behaviors[-1:]:
    required_behavior = RequiredBehavior(behavior_name=behavior[0])
    hazardous_behavior = HazardousBehavior(behavior_description=behavior[1])
    required_behavior.hazardous_behavior = hazardous_behavior

    session.add(required_behavior)

session.commit()

### Add test cases

In [ ]:
test_case_1 = TestCase()
tc_enhanced_concrete_scenario_1.test_case = test_case_1

test_case_1.required_behaviors.append(required_behavior_1)

session.add(test_case_1)
session.add(tc_enhanced_concrete_scenario_1)
session.commit()

### Try querying

In [ ]:
# Query for a specific FunctionalNominalScenario item (replace 'functional_nominal_scenario_id' with the actual ID of the FunctionalNominalScenario item)
functional_nominal_scenario_item = session.query(FunctionalNominalScenario).where(FunctionalNominalScenario.id == 2).first()

# If functional_nominal_scenario_item is found, traverse the relationship to ConcreteNominalScenario items
if functional_nominal_scenario_item:
    # Query ConcreteNominalScenario items related to the specific FunctionalNominalScenario item
    concrete_nominal_scenario_items = session.query(ConcreteNominalScenario).join(LogicalNominalScenario).join(FunctionalNominalScenario).filter(FunctionalNominalScenario.id == functional_nominal_scenario_item.id).all()

    # Now you have the ConcreteNominalScenario items related to the specific FunctionalNominalScenario item
    for concrete_nominal_scenario_item in concrete_nominal_scenario_items:
        print(concrete_nominal_scenario_item.path)
else:
    print("FunctionalNominalScenario item not found.")

In [ ]:

required_behavior_1 = RequiredBehavior(behavior_name="ego_drive_normally", behavior_description="Ego shall drive normally without harsh brake, keep same lane, reach destination at the end.")
required_behavior_2 = RequiredBehavior(behavior_name="ego_overtake_safe_lateral_distance", behavior_description="Ego shall keep safe lateral distance when overtaking road users.")
required_behavior_3 = RequiredBehavior(behavior_name="ego_safe_overtake_oncoming_traffic", behavior_description="Ego shall only overtake when time for manuevor is not enough, given that opposite lane has oncoming traffic.")
required_behavior_4 = RequiredBehavior(behavior_name="safe_longitudinal_distance", behavior_description="Ego shall keep safe longitudinal distance.")
required_behavior_5 = RequiredBehavior(behavior_name="ego_overtake", behavior_description="ego shall not overtake.")

hazardous_behavior_1 = HazardousBehavior(behavior_description="Ego caused collision.")
hazardous_behavior_2 = HazardousBehavior(behavior_description="Ego exceeded rated acceleration.")
hazardous_behavior_3 = HazardousBehavior(behavior_description="Ego violated minimum distance.")
hazardous_behavior_4 = HazardousBehavior(behavior_description="Ego violated safe lateral distance.")
hazardous_behavior_5 = HazardousBehavior(behavior_description="Ego overtaked unsafely.")

required_behavior_1.hazardous_behavior = hazardous_behavior_1
required_behavior_2.hazardous_behavior = hazardous_behavior_2
required_behavior_3.hazardous_behavior = hazardous_behavior_3
required_behavior_4.hazardous_behavior = hazardous_behavior_4
required_behavior_5.hazardous_behavior = hazardous_behavior_5

session.add(required_behavior_1)
session.add(required_behavior_2)
session.add(required_behavior_3)
session.add(required_behavior_4)
session.add(required_behavior_5)
session.commit()

In [ ]:
# Nominal
functional_nominal_scenario_2 = FunctionalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/functional_scenario_description.md")
# 
logical_nominal_scenario_2 = LogicalNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/logical_scenario_1.py")
# 
concrete_nominal_scenario_2 = ConcreteNominalScenario(path="nominal_scenario_catalog/nominal_scenario_2/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py")
# 
functional_nominal_scenario_2.logical_nominal_scenario_items.append(logical_nominal_scenario_2)
logical_nominal_scenario_2.concrete_nominal_scenario_items.append(concrete_nominal_scenario_2)
# 
session.add(functional_nominal_scenario_2)
session.commit()

# TC enhanced
# /home/weidong/Tools/Scenic/scenic_projects/tc_scenario_catalog/Cyclist_drives_on_opposite_lane/logical_scenario_1/concrete_scenario_1/concrete_scenario_1.py
tc_enhanced_functional_scenario_2 = TCEnhancedFunctionalScenario(path = "tc_scenario_catalog/Cyclist_drives_on_opposite_lane/functional_scenario_description.md",
                                                                 exist_non_complient = 0,
                                                                 functional_nominal_scenario_id = 2,
                                                                 triggering_condition_id = 14
                                                                 )
tc_enhanced_logical_scenario_2 = TCEnhancedLogicalScenario(path = "tc_scenario_catalog/Cyclist_drives_on_opposite_lane/logical_scenario_1/logical_scenario_1.py",
                                                           logical_nominal_scenario_id = 2)

tc_enhanced_functional_scenario_2.tc_enhanced_logical_scenario_items.append(tc_enhanced_logical_scenario_2)

session.add(tc_enhanced_functional_scenario_2)
session.commit()

# Test case
test_case_1 = TestCase()
tc_enhanced_concrete_scenario_1.test_case = test_case_1

test_case_1.required_behaviors.append(required_behavior_1)

session.add(test_case_1)
session.add(tc_enhanced_concrete_scenario_1)
session.commit()